# Import packages

In [ ]:
# import segmenteverygrain as seg

import sys
sys.path.append('/Users/pjlohr/Documents/GitHub/segmenteverygrain/segmenteverygrain')
import segmenteverygrain as seg

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import measure
from skimage.measure import regionprops, regionprops_table
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from importlib import reload
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from tqdm import trange, tqdm

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
gpus

# %matplotlib widget
# %matplotlib qt


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Load models

In [2]:
model = seg.Unet()
model.compile(optimizer=Adam(), loss=seg.weighted_crossentropy, metrics=["accuracy"])
model.load_weights('/Users/pjlohr/Documents/GitHub/segmenteverygrain/segmenteverygrain/checkpoints/seg_model')

sam_checkpoint = "/Users/pjlohr/Documents/GitHub/segmenteverygrain/segmenteverygrain/checkpoints/sam_vit_h_4b8939.pth"
device = "mps"
model_type = "default"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

2025-07-03 15:06:40.749979: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-07-03 15:06:40.750049: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-07-03 15:06:40.750053: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2025-07-03 15:06:40.750071: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-03 15:06:40.750080: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ImportError: `keras.optimizers.legacy` is not supported in Keras 3. When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy` optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable `TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.

# Run segmentation

### Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

### The segmentation can take a few minutes even for medium-sized images. Images with ~2000 pixels along their largest dimension are a good start and allow the user to get an idea about how well the segmentation works.

### If you have a much larger image, see the section **"Run segmentation on large image"** at the end of the notebook. Running the `predict_large_image` function takes a lot longer (e.g., several hours), but it is possible to analyze very large images with tens of thousands of grains.

### Image used below is available from [here](https://github.com/zsylvester/segmenteverygrain/blob/main/torrey_pines_beach.jpeg).

In [ ]:
import os
# reload(seg)

# replace this with the path to your image:
fname = '/Users/pjlohr/Documents/GitHub/segmenteverygrain/perovskite_segmentation/rapid/Matt_241113/0.4M/100C/50mmin_20percDMSO_25.4um_gap/center_m003_image.png'

data_dir = os.path.dirname(fname)
scale_factor = 2

image = np.array(load_img(path=fname))

# # Scale down image for much faster processing:
# image = np.array(load_img(path=fname, 
#                           color_mode='rgb',
#                           target_size=(image.shape[0] // scale_factor, image.shape[1] // scale_factor),
#                           interpolation='lanczos'
#                           ))

print(image.shape)  # Verify the shape (height, width, channels)

image_pred = seg.predict_image(image, model, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
labels, coords = seg.label_grains(image, image_pred, dbs_max_dist=1000.0) # Unet prediction

### Use the figure created in the next cell to check the quality of the Unet labeling (sometimes it doesn't work at all) and the distribution of SAM prompts (= black dots). If the Unet prediction is of poor quality, it is a good idea to create some training data and fine tune the base model so that it works better with the images of interest.

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.imshow(image)
plt.scatter(np.array(coords)[:,0], np.array(coords)[:,1], c='k')
plt.xticks([])
plt.yticks([]);


# Run SAM segmentation  

In [ ]:
# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, image, image_pred, 
            coords, labels, min_area=2000.0, plot_image=False, remove_edge_grains=True, remove_large_objects=False)


# plot results again if necessary
fig, ax = plt.subplots(figsize=(10,8))
ax.imshow(image)
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap='Paired')
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)

# Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [ ]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
                              lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [ ]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'labels' array after deleting and merging grains (the 'all_grains' list is updated when doing the deletion and merging):

In [ ]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

Plot the updated set of grains:

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap='Paired', plot_image=True)
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0]);

## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [ ]:
predictor = SamPredictor(sam)
predictor.set_image(image) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.onclick(event, ax, coords, image, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event', lambda event: seg.onpress(event, ax, fig))


In [ ]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [ ]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image, plotting=False)
fig, ax = plt.subplots(figsize=(10,8))
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap='Paired', plot_image=True)
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0])

## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [ ]:
cid5 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.click_for_scale(event, ax))

Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [ ]:
n_of_units = 200 # Units of length on scale bar. Typically use the entire bar for more precision
n_of_pixels = 1008
units_per_pixel = n_of_units/n_of_pixels # length of scale bar in pixels


In [ ]:
# Make directories
# os.makedirs(training_data_dir + "/grain_data/", exist_ok=True)
# os.makedirs(training_data_dir + "/colorful_grains/", exist_ok=True)
# os.makedirs(training_data_dir + "/masks/", exist_ok=True)
# os.makedirs(training_data_dir + "/labels/", exist_ok=True)

props = regionprops_table(labels.astype('int'), intensity_image = image, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)

stats_df = pd.DataFrame()

stats_df['area'] = grain_data['area'].values * units_per_pixel**2
stats_df['perimeter'] = grain_data['perimeter'].values * units_per_pixel
stats_df['major_axis_length'] = grain_data['major_axis_length'].values * units_per_pixel
stats_df['minor_axis_length'] = grain_data['minor_axis_length'].values * units_per_pixel
stats_df['orientation'] = grain_data['orientation'].values

stats_df.to_csv(data_dir + '/' +fname.split('/')[-1][:-4]+'.csv') # save grain data to CSV file
# stats_df.head()

# Get colorful grains
fig, ax = plt.subplots(figsize=(10,8))
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap='plasma', plot_image=True, color_by_size=True)
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0])
# Save the plot with minimal white space
plt.savefig(data_dir + '/' +fname.split('/')[-1][:-4] + '_colorful_grains.png', bbox_inches='tight', pad_inches=0)
# write grayscale mask to PNG file
cv2.imwrite(data_dir + '/' +fname.split('/')[-1][:-4] + '_mask.png', mask_all)

# Define a colormap using matplotlib
num_classes = len(all_grains)
cmap = plt.get_cmap('plasma', num_classes)
# Map each class label to a unique color using the colormap
vis_mask = cmap(labels.astype(np.uint16))[:,:,:3] * 255
vis_mask = vis_mask.astype(np.uint8)

# Save the colored labels as a PNG file
cv2.imwrite(data_dir + '/' + fname.split('/')[-1][:-4] + '_labels.png', vis_mask)

# Save the image as a PNG file
cv2.imwrite(data_dir + '/' + fname.split('/')[-1][:-4] + '_image.png', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


### Finetuning the base model

In [ ]:
# # patchify images and masks
# input_dir = "./Masks_and_images/" # the input directory should contain files with 'image' and 'mask' in their filenames
# patch_dir = "./New_project/" # a directory called "Patches" will be created here
# image_dir, mask_dir = seg.patchify_training_data(input_dir, patch_dir)

In [ ]:
# # create training, validation, and test datasets
# train_dataset, val_dataset, test_dataset = seg.create_train_val_test_data(image_dir, mask_dir, augmentation=True)

In [ ]:
# # load base model weights and train the model with the new data
# weights_dir = './checkpoints/seg_model'
# model = seg.create_and_train_model(weights_dir, train_dataset, val_dataset, test_dataset, epochs=100)

In [ ]:
# # save finetuned model as new model (this then can be loaded using "model.load_weights('./new_model/seg_model')"
# model.save_weights('./new_model/seg_model')